In [1]:
from time import time
from glob import glob
from typing import Tuple, List

import numpy as np
import pandas as pd
import networkx as nx

In [2]:
def euclidean2D_distance(u: Tuple, v: Tuple) -> int:
    du = u[0] - u[1]
    dv = v[0] - v[1]

    rounded_distance = np.round(np.sqrt(du**2 + dv**2))
    return int(rounded_distance)

def pseudo_eclidean2D_distance(u: Tuple, v: Tuple) -> int:
    du = u[0] - u[1]
    dv = v[0] - v[1]

    real_distance = np.sqrt((du**2 + dv**2) / 10)
    rounded_distance = np.round(real_distance)

    if rounded_distance < real_distance:
        return rounded_distance + 1

    return int(rounded_distance)


class TSPInstance:
    def __init__(self, filename: str, debug: bool = False) -> None:
        with open(filename, 'r') as f:
            line = f.readline()

            # Processando o cabeçalho do arquivo
            self._header = dict()
            while line.strip() != 'NODE_COORD_SECTION':
                key = line.split(':')[0].strip()
                self._header[key] = ' '.join(line.split(':')[1:]).strip()

                if key == 'DIMENSION':
                    self._header[key] = int(self._header[key])

                line = f.readline()

            self._validate_header()
            if debug:
                self._log_header()

            # Processando as coordenadas do arquivo
            self._coords = list()
            for _ in range(self._header['DIMENSION']):
                _, x, y = f.readline().split()
                self._coords.append((float(x), float(y)))

            # Definindo a função de distância
            self._dist_function = euclidean2D_distance
            if self._header['EDGE_WEIGHT_TYPE'] == 'ATT':
                self._dist_function = pseudo_eclidean2D_distance

    def get_instance_as_graph(self) -> nx.Graph:
        graph = nx.Graph()
        n_nodes = self._header['DIMENSION']

        for i in range(0, n_nodes):
            for j in range(i+1, n_nodes):
                graph.add_edge(i, j, weight=self._dist_function(self._coords[i], self._coords[j]))

        return graph

    def get_instance_as_adj_matrix(self) -> np.ndarray:
        n_nodes = self._header['DIMENSION']
        matrix = np.zeros((n_nodes, n_nodes))

        for i in range(0, n_nodes):
            for j in range(i+1, n_nodes):
                matrix[i][j] = self._dist_function(self._coords[i], self._coords[j])
                matrix[j][i] = self._dist_function(self._coords[j], self._coords[i])
        
        # Um vértice não possui conexões com ele mesmo!
        # Logo, a distância deve ser infinita
        np.fill_diagonal(matrix, np.inf)

        return matrix

    def _validate_header(self) -> None:
        if 'TYPE' not in self._header:
                raise KeyError('TYPE key missing from file')
        if 'DIMENSION' not in self._header:
                raise KeyError('DIMENSION key missing from file')
        if 'EDGE_WEIGHT_TYPE' not in self._header:
                raise KeyError('EDGE_WEIGHT_TYPE key missing from file')
        if self._header['TYPE'] != 'TSP':
            raise ValueError('Instance type must be TSP')
        if self._header['EDGE_WEIGHT_TYPE'] not in ('EUC_2D', 'ATT'):
            raise ValueError('EDGE_WEIGHT_TYPE must be either EUC_2D or ATT')

    def _log_header(self) -> None:
        print('File self._header information:')
        for key in self._header:
            print(' - {}: {}'.format(key, self._header[key]))

def get_nearest_neighbor(v: int, instance: np.ndarray, solution: List[int]) -> int:
    min_value = np.inf
    nearest_neighbor = -1
    
    for i in range(len(instance)):
        if i not in solution and instance[v][i] < min_value:
            min_value = instance[v][i]
            nearest_neighbor = i

    return nearest_neighbor

def heuristic(instance: np.ndarray, v: int = 0) -> List[int]:
    solution = [v]
    for _ in range(len(instance) - 1):
        next_node = get_nearest_neighbor(v, instance, solution)
        solution.append(next_node)
        v = next_node

    # Adicionando a raiz para completar o ciclo
    return solution + [solution[0]]

def compute_path_cost(instance: np.ndarray, nodes: List[int]) -> float:
    cost = 0.0
    for i in range(len(nodes) - 1):
        cost += instance[nodes[i]][nodes[i+1]]

    return cost

n_iter = 5
files = glob('../tsp_instances/*.tsp') + glob('../tsp_instances/EUC_2D/*.tsp')

data = {'instância': [], 'tempo de execução': [], 'número de cidades': [], 'custo da solução': [], 'distância utilizada': []}
for f in files:
    begin = time()
    for _ in range(n_iter):
        instance = TSPInstance(filename=f).get_instance_as_adj_matrix()
        solution = heuristic(instance)

    mean_time = (time() - begin) / n_iter
    distance_type = 'pseudo-euclidiana' if 'att' in f else 'euclidiana 2D'
    
    data['instância'].append(f.split('/')[-1])
    data['número de cidades'].append(len(instance))
    data['tempo de execução'].append(np.round(mean_time, 2))
    data['custo da solução'].append(compute_path_cost(instance, solution))
    data['distância utilizada'].append(distance_type)

In [3]:
df = pd.DataFrame(data).sort_values('tempo de execução').reset_index(drop=True)
df

,instância,tempo de execução,número de cidades,custo da solução,distância utilizada
0,att48.tsp,0.02,48,62956.0,pseudo-euclidiana
1,berlin52.tsp,0.02,52,28503.0,euclidiana 2D
2,st70.tsp,0.03,70,3151.0,euclidiana 2D
3,pr76.tsp,0.04,76,542696.0,euclidiana 2D
4,kroC100.tsp,0.07,100,176786.0,euclidiana 2D
5,lin105.tsp,0.07,105,167288.0,euclidiana 2D
6,kroE100.tsp,0.07,100,207322.0,euclidiana 2D
7,kroD100.tsp,0.07,100,178418.0,euclidiana 2D
8,kroB100.tsp,0.07,100,202410.0,euclidiana 2D
9,kroA100.tsp,0.07,100,188069.0,euclidiana 2D
